In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import mean_absolute_error

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import category_encoders as ce

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [2]:
# error_bad_lines: drop lines that have too many columns
df = pd.read_csv("../data/tc20171021.csv", error_bad_lines=False, verbose=False)

b'Skipping line 297911: expected 9 fields, saw 10\nSkipping line 297912: expected 9 fields, saw 10\nSkipping line 297913: expected 9 fields, saw 10\nSkipping line 297914: expected 9 fields, saw 10\nSkipping line 297915: expected 9 fields, saw 10\nSkipping line 297916: expected 9 fields, saw 10\nSkipping line 297917: expected 9 fields, saw 10\nSkipping line 297918: expected 9 fields, saw 10\nSkipping line 297919: expected 9 fields, saw 10\nSkipping line 297920: expected 9 fields, saw 10\nSkipping line 297921: expected 9 fields, saw 10\nSkipping line 297922: expected 9 fields, saw 10\nSkipping line 297923: expected 9 fields, saw 10\nSkipping line 297925: expected 9 fields, saw 10\nSkipping line 297926: expected 9 fields, saw 10\nSkipping line 297927: expected 9 fields, saw 10\nSkipping line 297928: expected 9 fields, saw 10\nSkipping line 297929: expected 9 fields, saw 10\nSkipping line 297930: expected 9 fields, saw 10\nSkipping line 297931: expected 9 fields, saw 10\nSkipping line 2979

In [3]:
df.shape()
df.head()

TypeError: 'tuple' object is not callable

In [8]:
df.describe()

,id,price,year,mileage
count,1.216250e+06,1.216250e+06,1.216250e+06,1.216250e+06
mean,6.132814e+05,2.096887e+04,2.013307e+03,5.290810e+04
std,3.546571e+05,1.315940e+04,3.441842e+00,8.192775e+04
min,1.000000e+00,1.500000e+03,1.997000e+03,5.000000e+00
25%,3.081122e+05,1.299400e+04,2.012000e+03,2.396200e+04
50%,6.134915e+05,1.799500e+04,2.014000e+03,4.053400e+04
75%,9.175538e+05,2.641400e+04,2.016000e+03,7.272500e+04
max,1.233042e+06,4.995000e+05,2.018000e+03,7.758776e+07


In [6]:
#Lowers all column names
df.columns = map(str.lower, df.columns)

In [7]:
df.head()

,id,price,year,mileage,city,state,vin,make,model
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


In [45]:
df[df["make"] == "Ford"]

,id,price,year,mileage,city,state,vin,make,model
1199,1200,9799,2012,37419,San Antonio,TX,3FAHP0HAXCR189181,Ford,Fusion4dr
1200,1201,13585,2016,35825,Oswego,NY,3FA6P0H79GR212404,Ford,FusionSE
1201,1202,11810,2015,46872,Alexandria,VA,3FA6P0G78FR102170,Ford,FusionS
1202,1203,10925,2015,19279,Raleigh,NC,1FADP3E29FL353911,Ford,FocusSedan
1203,1204,12926,2016,12055,Gaithersburg,MD,1FADP3K24GL357113,Ford,FocusHatchback
...,...,...,...,...,...,...,...,...,...
708595,713962,299999,2005,8718,Vista,CA,1FAFP90S35Y401617,Ford,GTCoupe
708596,713963,299000,2005,4479,Westville,NJ,1FAFP90S15Y401390,Ford,GTCoupe
708597,713964,299995,2006,2407,Glen Ellyn,IL,1FAFP90S36Y400579,Ford,GTCoupe
708598,713965,379888,2006,6215,Boerne,TX,1FAFP90S36Y400291,Ford,GTCoupe


In [5]:
df2 = pd.read_csv("../data/vehicles.csv")

/Users/mph/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df2.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1300,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,2450,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-7250,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1000,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,2450,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-7250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2000,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-5000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


## Helper Functions


In [13]:
def subset(df):
    """Function to help pull relevant makes from loaded datasets"""
    
    df = df.copy()
    
    df.columns = map(str.lower, df.columns)
    
#     df = df[df["make"] == col]
    df["make"] = df["make"].apply(lambda x: x.lower())
    df["model"] = df["model"].apply(lambda x: x.lower())
    df["state"] = df["state"].apply(lambda x: x.lower())
    
    return df

In [8]:
test = subset(df, "Chevrolet")
samp = test[["make", "model"]]
print(samp.shape)
samp.head()

(150426, 2)


,make,model
905,chevrolet,cruzels
906,chevrolet,malibu
907,chevrolet,malibu1lt
908,chevrolet,malibuls
909,chevrolet,malibult


In [9]:
test2 = subset (df2, "Chevrolet")
samp2 = test2[["make", "model"]]
print(samp2.shape)
samp2[samp2["model"] == "malibu"].head()

(4086, 2)


,make,model
3944,chevrolet,malibu
4774,chevrolet,malibu
4775,chevrolet,malibu
5654,chevrolet,malibu
5655,chevrolet,malibu


In [10]:
pd.merge(test, test2, on=["make","model", "year"])

,id_x,price,year,mileage,city,state,vin,make,model,barrels08,barrelsa08,charge120,charge240,city08,city08u,citya08,citya08u,citycd,citye,cityuf,co2,co2a,co2tailpipeagpm,co2tailpipegpm,comb08,comb08u,comba08,comba08u,combe,combinedcd,combineduf,cylinders,displ,drive,engid,eng_dscr,fescore,fuelcost08,fuelcosta08,fueltype,fueltype1,ghgscore,ghgscorea,highway08,highway08u,highwaya08,highwaya08u,highwaycd,highwaye,highwayuf,hlv,hpv,id_y,lv2,lv4,mpgdata,phevblended,pv2,pv4,range,rangecity,rangecitya,rangehwy,rangehwya,trany,ucity,ucitya,uhighway,uhighwaya,vclass,yousavespend,guzzler,trans_dscr,tcharger,scharger,atvtype,fueltype2,rangea,evmotor,mfrcode,c240dscr,charge240b,c240bdscr,createdon,modifiedon,startstop,phevcity,phevhwy,phevcomb
0,907,10900,2011,18001,Belton,MO,1G1ZA5EU5BF368258,chevrolet,malibu,12.677308,0.000000,0.0,0.0,22,21.8240,0,0.0000,0.0,0.0,0.0,-1,-1,0.000000,341.807692,26,25.5855,0,0.0000,0.0,0.0,0.0,4.0,2.4,Front-Wheel Drive,26,NaN,-1,1050,0,Regular,Regular Gasoline,-1,-1,32,32.4136,0,0.0000,0.0,0.0,0.0,0,0,30380,0,16,Y,False,0,95,0,0.0,0.0,0.0,0.0,Automatic (S6),27.8000,0.0,46.8000,0.0,Midsize Cars,-250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Mon Sep 26 00:00:00 EDT 2016,NaN,0,0,0
1,907,10900,2011,18001,Belton,MO,1G1ZA5EU5BF368258,chevrolet,malibu,16.480500,0.000000,0.0,0.0,17,16.9179,0,0.0000,0.0,0.0,0.0,-1,-1,0.000000,444.350000,20,19.9022,0,0.0000,0.0,0.0,0.0,6.0,3.6,Front-Wheel Drive,68,NaN,-1,1350,0,Regular,Regular Gasoline,-1,-1,25,25.3726,0,0.0000,0.0,0.0,0.0,0,0,30384,0,16,N,False,0,95,0,0.0,0.0,0.0,0.0,Automatic (S6),21.0858,0.0,35.7384,0.0,Midsize Cars,-1750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Mon Sep 26 00:00:00 EDT 2016,NaN,0,0,0
2,907,10900,2011,18001,Belton,MO,1G1ZA5EU5BF368258,chevrolet,malibu,12.677308,4.406471,0.0,0.0,22,21.8240,15,14.6017,0.0,0.0,0.0,-1,-1,370.117647,341.807692,26,25.6029,17,17.3974,0.0,0.0,0.0,4.0,2.4,Front-Wheel Drive,149,FFV,-1,1050,2000,Gasoline or E85,Regular Gasoline,-1,-1,32,32.4757,23,22.7122,0.0,0.0,0.0,0,0,30392,0,16,Y,False,0,95,0,0.0,0.0,0.0,0.0,Automatic (S6),27.8000,18.6,46.9000,32.8,Midsize Cars,-250,NaN,NaN,NaN,NaN,FFV,E85,280,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Mon Sep 26 00:00:00 EDT 2016,NaN,0,0,0
3,920,14989,2016,18576,Brook Park,OH,1G11B5SA4GF133927,chevrolet,malibu,12.677308,0.000000,0.0,0.0,22,22.1809,0,0.0000,0.0,0.0,0.0,342,-1,0.000000,342.000000,26,25.8180,0,0.0000,0.0,0.0,0.0,4.0,2.0,Front-Wheel Drive,186,SIDI,6,1050,0,Regular,Regular Gasoline,6,-1,32,32.2893,0,0.0000,0.0,0.0,0.0,0,0,37138,13,13,Y,False,100,100,0,0.0,0.0,0.0,0.0,Automatic (S8),28.3000,0.0,46.6000,0.0,Midsize Cars,-250,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,GMX,NaN,0.0,NaN,Thu Nov 19 00:00:00 EST 2015,Mon Sep 26 00:00:00 EDT 2016,N,0,0,0
4,920,14989,2016,18576,Brook Park,OH,1G11B5SA4GF133927,chevrolet,malibu,10.987000,0.000000,0.0,0.0,27,26.7185,0,0.0000,0.0,0.0,0.0,293,-1,0.000000,293.000000,30,30.2730,0,0.0000,0.0,0.0,0.0,4.0,1.5,Front-Wheel Drive,180,SIDI,7,900,0,Regular,Regular Gasoline,7,-1,36,36.1511,0,0.0000,0.0,0.0,0.0,0,0,37146,13,13,Y,False,100,100,0,0.0,0.0,0.0,0.0,Automatic 6-spd,34.8000,0.0,52.9000,0.0,Midsize Cars,500,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,GMX,NaN,0.0,NaN,Wed Dec 02 00:00:00 EST 2015,Mon Sep 26 00:00:00 EDT 2016,Y,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10006,502477,29888,2015,19433,Plano,TX,2G1FKJEJ9F9236505,chevrolet,camaro,15.695714,0.000000,0.0,0.0,18,17.6611,0,0.0000,0.0,0.0,0.0,428,-1,0.000000,428.000000,21,20.7653,0,0.0000,0.0,0.0,0.0,6.0,3.6,Rear-Wheel Drive,86,SIDI,5,1300,0,Regular,Regular Gasoline,5,-1,26,26.4466,0,0.0000,0.0,0.0,0.0,0,0,35216,11,0,N,Fals

In [11]:
df[df["Make"]=="Chevrolet"]

,Id,Price,Year,Mileage,City,State,Vin,Make,Model
905,906,10700,2015,24236,Princeton,NJ,1G1PB5SG6F7285330,Chevrolet,CruzeLS
906,907,10900,2011,18001,Belton,MO,1G1ZA5EU5BF368258,Chevrolet,Malibu
907,908,14500,2014,12519,St Louis,MO,1G11C5SL4EF174599,Chevrolet,Malibu1LT
908,909,9977,2014,70561,Villa Park,IL,1G11B5SL7EF106817,Chevrolet,MalibuLS
909,910,17233,2017,25076,Fort Lauderdale,FL,1G1ZE5ST1HF149502,Chevrolet,MalibuLT
...,...,...,...,...,...,...,...,...,...
540580,545947,99999,2007,233587,Rockford,MI,3GNDA13D17S595664,Chevrolet,HHR2WD
540581,545948,99998,2003,121000,Rockford,MI,1GNFG15T231118331,Chevrolet,Express
540582,545949,99998,2003,123560,Rockford,MI,3GNEK13T33G114352,Chevrolet,Avalanche1500
540583,545950,99999,2002,264000,Rockford,MI,1GNCT18W72K101098,Chevrolet,Blazer2dr


In [12]:
df2[df2['make'] == "Chevrolet"]

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
35,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,4122,(FFS),-1,1300,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,10029,15,16,Chevrolet,Lumina,N,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 3-spd,23.0000,0.0,37.0000,0.0,Midsize Cars,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
37,16.480500,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.1,Front-Wheel Drive,4117,(FFS),-1,1350,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,10030,15,16,Chevrolet,Lumina,Y,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 3-spd,21.0000,0.0,35.0000,0.0,Midsize Cars,1993,-1750,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
38,16.480500,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.1,Front-Wheel Drive,4118,(FFS),-1,1350,0,Regular,Regular Gasoline,-1,-1,27,0.0,0,0.0,0.0,0.0,0.0,0,0,10031,15,16,Chevrolet,Lumina,Y,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 4-spd,21.0000,0.0,38.0000,0.0,Midsize Cars,1993,-1750,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
39,18.311667,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,493.722222,18,0.0,0,0.0,0.0,0.0,0.0,6.0,3.4,Front-Wheel Drive,4120,(FFS),-1,1500,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,0.0,0.0,0.0,0,0,10032,15,16,Chevrolet,Lumina,N,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 4-spd,19.0000,0.0,33.0000,0.0,Midsize Cars,1993,-2500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
40,18.311667,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,493.722222,18,0.0,0,0.0,0.0,0.0,0.0,6.0,3.4,Front-Wheel Drive,4119,(FFS),-1,1500,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,0.0,0.0,0.0,0,0,10033,15,16,Chevrolet,Lumina,N,False,95,100,0,0.0,0.0,0.0,0.0,Manual 5-spd,18.8889,0.0,34.0000,0.0,Midsize Cars,1993,-2500,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42142,23.543571,0.0,0.0,0.0,12,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,634.785714,14,0.0,0,0.0,0.0,0.0,0.0,8.0,5.0,Rear-Wheel Drive,4860,(GM-CHEV),-1,1950,0,Regular,Regular Gasoline,-1,-1,16,0.0,0,0.0,0.0,0.0,0.0,0,0,992,0,0,Chevrolet,G10/20 Sport Van 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 4-spd,15.0000,0.0,22.0000,0.0,Vans,1985,-4750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42153,25.354615,0.0,0.0,0.0,11,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,683.615385,13,0.0,0,0.0,0.0,0.0,0.0,8.0,5.7,Rear-Wheel Drive,4870,(350 V8),-1,2100,0,Regular,Regular Gasoline,-1,-1,16,0.0,0,0.0,0.0,0.0,0.0,0,0,993,0,0,Chevrolet,G10/20 Sport Van 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Au

In [13]:
sub = df2[df2["make"] == "Chevrolet"]
sub = sub[["make", "model","year", "co2", "comb08"]]
sub["silverado"] = sub["model"].apply(lambda x: x.find("Silverado"))
sub = sub[sub["silverado"] == 0]
sub[sub["year"] == 2010]

,make,model,year,co2,comb08,silverado
20599,Chevrolet,Silverado C15 2WD,2010,-1,17,0
20608,Chevrolet,Silverado K15 4WD,2010,-1,15,0
21728,Chevrolet,Silverado C15 2WD,2010,-1,16,0
21729,Chevrolet,Silverado C15 2WD,2010,-1,17,0
21730,Chevrolet,Silverado C15 2WD,2010,-1,15,0
21731,Chevrolet,Silverado 15 Hybrid 2WD,2010,-1,22,0
21732,Chevrolet,Silverado C15 XFE 2WD,2010,-1,18,0
21739,Chevrolet,Silverado K15 4WD,2010,-1,15,0
21740,Chevrolet,Silverado K15 4WD,2010,-1,17,0
21741,Chevrolet,Silverado K15 4WD,2010,-1,14,0


In [14]:
df.shape

(1216250, 9)

In [15]:
sub2 = df[df["Make"] == "Chevrolet"]
sub2["silverado"] = sub2["Model"].apply(lambda x: x.find("Silverado"))
sub2 = sub2[sub2["silverado"] == 0]
sub2[sub2["Year"] == 2010]

/Users/mph/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Id,Price,Year,Mileage,City,State,Vin,Make,Model,silverado
984,985,12500,2010,74936,Troy,MO,1GCPCPEX4AZ238628,Chevrolet,Silverado,0
1034,1035,22900,2010,59000,Lincoln,NE,3GCRKSE3XAG156978,Chevrolet,Silverado,0
93049,93050,28995,2010,115017,Baird,TX,1GC4KXB66AF101371,Chevrolet,Silverado,0
93431,93432,32995,2010,118622,Pelham,AL,1GB7K0B60AF127747,Chevrolet,Silverado,0
93967,93968,37499,2010,95631,Martinsburg,WV,1GC4K1B69AF113544,Chevrolet,Silverado,0
...,...,...,...,...,...,...,...,...,...,...
536278,541645,41878,2010,59170,Prattville,AL,1GC7K1B65AF129351,Chevrolet,Silverado,0
538040,543407,21795,2010,140749,Clinton,SC,3GCRKTE34AG164029,Chevrolet,Silverado,0
538160,543527,41400,2010,78669,Elko,NV,1GC4KYB65AF125716,Chevrolet,Silverado,0
538812,544179,44995,2010,56848,Fontana,CA,1GC4KYB63AF134429,Chevrolet,Silverado,0


In [16]:
df.head()

,Id,Price,Year,Mileage,City,State,Vin,Make,Model
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


In [17]:
df.describe()

,Id,Price,Year,Mileage
count,1.216250e+06,1.216250e+06,1.216250e+06,1.216250e+06
mean,6.132814e+05,2.096887e+04,2.013307e+03,5.290810e+04
std,3.546571e+05,1.315940e+04,3.441842e+00,8.192775e+04
min,1.000000e+00,1.500000e+03,1.997000e+03,5.000000e+00
25%,3.081122e+05,1.299400e+04,2.012000e+03,2.396200e+04
50%,6.134915e+05,1.799500e+04,2.014000e+03,4.053400e+04
75%,9.175538e+05,2.641400e+04,2.016000e+03,7.272500e+04
max,1.233042e+06,4.995000e+05,2.018000e+03,7.758776e+07


In [18]:
df.describe(exclude="number")

,City,State,Vin,Make,Model
count,1216250,1216250,1216250,1216250,1216250
unique,2599,59,1216250,58,3105
top,Houston,TX,2C3CCABG0DH533069,Ford,Silverado
freq,20315,139335,1,161999,36174


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216250 entries, 0 to 1216249
Data columns (total 9 columns):
Id         1216250 non-null int64
Price      1216250 non-null int64
Year       1216250 non-null int64
Mileage    1216250 non-null int64
City       1216250 non-null object
State      1216250 non-null object
Vin        1216250 non-null object
Make       1216250 non-null object
Model      1216250 non-null object
dtypes: int64(4), object(5)
memory usage: 83.5+ MB


In [20]:
def cleaner(df):
    df = df.copy()
    
    # Capitalize all values to reduce duplicates
    df["State"] = df["State"].apply(lambda x: x.upper())
    df["City"] = df["City"].apply(lambda x: x.upper())
    df["Model"] = df["Model"].apply(lambda x: x.upper())
    
    # Drop Noise
    cols = ["Id", "Vin"]
    df = df.drop(cols, axis=1)
    
    return df

df = cleaner(df)

In [21]:
df.head(1)

,Price,Year,Mileage,City,State,Make,Model
0,16472,2015,18681,JEFFERSON CITY,MO,Buick,ENCORECONVENIENCE


In [22]:
df.describe()

,Price,Year,Mileage
count,1.216250e+06,1.216250e+06,1.216250e+06
mean,2.096887e+04,2.013307e+03,5.290810e+04
std,1.315940e+04,3.441842e+00,8.192775e+04
min,1.500000e+03,1.997000e+03,5.000000e+00
25%,1.299400e+04,2.012000e+03,2.396200e+04
50%,1.799500e+04,2.014000e+03,4.053400e+04
75%,2.641400e+04,2.016000e+03,7.272500e+04
max,4.995000e+05,2.018000e+03,7.758776e+07


In [5]:
df.describe(exclude="number")

,City,State,Vin,Make,Model
count,1216250,1216250,1216250,1216250,1216250
unique,2599,59,1216250,58,3105
top,Houston,TX,4T3BK3BB4AU026732,Ford,Silverado
freq,20315,139335,1,161999,36174


In [19]:
# Create the preprocess pipe

pre_pipe = make_pipeline(
    ce.OrdinalEncoder(),
    MinMaxScaler()    
)

df= subset(df)

In [20]:
df.head()

,id,price,year,mileage,city,state,vin,make,model
0,1,16472,2015,18681,Jefferson City,mo,KL4CJBSBXFB267643,buick,encoreconvenience
1,2,15749,2015,27592,Highland,in,KL4CJASB5FB245057,buick,encorefwd
2,3,16998,2015,13650,Boone,nc,KL4CJCSB0FB264921,buick,encoreleather
3,4,15777,2015,25195,New Orleans,la,KL4CJASB4FB217542,buick,encorefwd
4,5,16784,2015,22800,Las Vegas,nv,KL4CJBSB3FB166881,buick,encoreconvenience


In [21]:
# Split
training, test = train_test_split(df)
train, val = train_test_split(training)

target = "price"

X_train = train.drop([target, "city", "id", "vin"], axis=1)
y_train = train[target]

X_val = val.drop([target, "city", "id", "vin"], axis=1)
y_val= val[target]

X_test = test.drop([target, "city", "id", "vin"], axis=1)
y_test = test[target]

#Cross validation set
X_cv = training.drop([target, "city", "id", "vin"], axis=1)
y_cv = training[target]

In [23]:
X_train_process = pre_pipe.fit_transform(X_train)
print("done")
X_val_process = pre_pipe.transform(X_val)
print("done")
X_test_process = pre_pipe.transform(X_test)

done
done


In [62]:
X_test_process = pre_pipe.transform(X_test)

In [63]:
from numpy import savetxt

#Save the preprocess so we dont have to wait for the large amount of data to preprocess again >20mins on my computer

# savetxt("X_train_process.csv", X_train_process, delimiter=",")
# savetxt("X_val_process.csv", X_val_process, delimiter=",")
# savetxt("X_test_process.csv", X_test_process, delimiter=",")

# y_train.to_csv("y_train.csv")
# y_val.to_csv("y_val.csv")
# y_test.to_csv("y_test.csv")

/Users/mph/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':
/Users/mph/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/mph/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # This is added back b

In [20]:
X_train_process.shape

(684140, 6)

In [3]:
# xtrain_process = pd.read_csv("X_train_process.csv")
# xval_process = pd.read_csv("X_val_process.csv")
# xtest_process = pd.read_csv("X_test_process.csv")
# ytrain = pd.read_csv("y_train.csv")
# yval = pd.read_csv("y_val.csv")
# ytest = pd.read_csv("y_test.csv")

In [32]:
training.head()

,Price,Year,Mileage,City,State,Make,Model
968360,24595,2014,28006,WESTBURY,NY,Mercedes-Benz,C-CLASSC350
121965,20998,2014,33251,PARAMUS,NJ,Chevrolet,EQUINOXAWD
923488,4499,2002,136334,SNELLVILLE,GA,Lexus,RX
1121000,15000,2016,48532,TOMS RIVER,NJ,Toyota,CAMRYSE
115862,21699,2016,46812,CHESAPEAKE,VA,Chevrolet,IMPALA2LT


In [36]:
price_mean = training["Price"].mean()
y_true = training["Price"]

print(price_mean)

mean_absolute_error(y_true, [price_mean for x in range(len(y_true))])

20963.102260830292


8889.39171907388

In [40]:
def best_guess(x):
    return 30000-((2020 - x)*2000)

best_guess(2000)

-10000

In [42]:
training["best_guess"] = training["Year"].apply(best_guess)

/Users/mph/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
print(mean_absolute_error(y_true, training["best_guess"]))

print(r2_score(y_true, [price_mean for x in range(len(y_true))]))

from sklearn.metrics import r2_score

r2_score(y_true, training["best_guess"])

8039.67615631444
0.0


0.04095390534846166

In [ ]:
#Early Stopping
es = EarlyStopping(patience=10)

#Model
model = Sequential()

#Input
model.add(Dense(64, input_dim=5, activation='relu'))

#Hidden
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))



#Output
model.add(Dense(1))

#Compile
model.compile(loss='mean_absolute_error',
              metrics=['RootMeanSquaredError'],
              optimizer='adam')

#Fit & Evaluate
history = model.fit(X_train_process, 
                    y_train, 
                    epochs=1000, 
                    verbose=True,
                    validation_data=(X_val_process, y_val),
                    callbacks=[es])

Train on 684140 samples, validate on 228047 samples
Epoch 1/1000
684140/684140 [==============================] - 33s 48us/sample - loss: 7542.9108 - RootMeanSquaredError: 12360.0664 - val_loss: 7287.5862 - val_RootMeanSquaredError: 11819.2861
Epoch 2/1000
684140/684140 [==============================] - 33s 49us/sample - loss: 7213.7520 - RootMeanSquaredError: 11805.4219 - val_loss: 7115.9309 - val_RootMeanSquaredError: 11608.3525
Epoch 3/1000
684140/684140 [==============================] - 38s 56us/sample - loss: 7114.0381 - RootMeanSquaredError: 11651.4902 - val_loss: 7048.2598 - val_RootMeanSquaredError: 11530.9365
Epoch 4/1000
684140/684140 [==============================] - 41s 61us/sample - loss: 7066.4259 - RootMeanSquaredError: 11603.9014 - val_loss: 7090.8493 - val_RootMeanSquaredError: 11781.2412
Epoch 5/1000
684140/684140 [==============================] - 41s 60us/sample - loss: 7016.3899 - RootMeanSquaredError: 11560.9258 - val_loss: 6930.9642 - val_RootMeanSquaredError:

In [38]:
model.save("model.h5")

In [34]:
print(history.history.keys())

dict_keys(['loss', 'RootMeanSquaredError', 'val_loss', 'val_RootMeanSquaredError'])


In [39]:
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [60]:
#Model
model = Sequential()

#Input
model.add(Dense(16, input_dim=276, activation='relu'))

#Hidden
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))

#Output
model.add(Dense(1, activation='linear'))

#Compile
model.compile(loss='mean_squared_error',
              metrics=['mean_squared_error'],
              optimizer='adam')

#Fit & Evaluate
history = model.fit(X_train, y_train, epochs=10, verbose=True, validation_split=.1)
scores = model.evaluate(X_test, y_test, verbose=0)

ValueError: Error when checking input: expected dense_input to have shape (276,) but got array with shape (6,)

In [ ]:
print(f"The MSE of our neural net is ${scores[1]}")
print(f"The RMSE of our neural net is ${round(np.sqrt(scores[1]), 2)}")